In [231]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
from urllib.parse import urljoin
import re

In [232]:
driver =  webdriver.Chrome(executable_path = 'D:\workspace\chromedriver.exe' )

driver.get('https://ev.or.kr/evmonitor')
time.sleep(5)

In [233]:
sid = []

In [234]:
for k in range(2,19):
    driver.find_element_by_css_selector('#F_SI_DO1')
    driver.implicitly_wait(5)
    driver.find_element_by_css_selector('#F_SI_DO1 > option:nth-child(%s)'%k).click()
    time.sleep(3)
    driver.find_element_by_css_selector('#statList')
    driver.implicitly_wait(30)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    a = soup.find_all('div',class_='fw_path')
    for link in a:
        temp = str(link['sid'])
        sid.append(temp)

In [235]:
len(sid)

2491

In [236]:
name =[]  #이름
add = []  #주소
time =[]  #사용가능시간
typ = []  # 구분
agency = [] # 운영기관
phone = [] # 연락처
fee = [] # 충전요금
amount = [] # 급속충전량
etc = [] # 참고사항
nouse = [] # 운전상태
using = []
ben = []
charge = []

In [237]:
for i in sid:
    url_base = 'https://www.ev.or.kr/portal/monitor/evminfo?sid='
    url = url_base+str(i)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    
    n_tmp = soup.find_all('h2')
    name.append(n_tmp[0].get_text())
    
    t_tmp = soup.find_all('em')
    time.append(t_tmp[0].get_text())
    
    ty_tmp = soup.find_all('dd')
    typ.append(ty_tmp[0].get_text())

    info_tmp = soup.find_all(class_='table_02')
    dinfo = info_tmp[0].find_all('td')
    add.append(dinfo[0].get_text())
    agency.append(dinfo[1].get_text())
    phone.append(dinfo[2].get_text())
    tmp_fee = dinfo[3].get_text()
    tmp_fee = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_fee)
    fee.append(tmp_fee)
    tmp_amount = dinfo[4].get_text()
    tmp_amount = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_amount)
    amount.append(tmp_amount)
    tmp_etc = dinfo[5].get_text()
    tmp_etc = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_etc)
    etc.append(tmp_etc)
    no=0
    use=0
    den=0
    for tmp_using in soup.find_all(class_='td3'):
        a = tmp_using.find_all('span')
        tmp = a[0].get_text()
        if tmp =='사용가능':
            no +=1
        elif tmp == '사용중':
            use +=1
        else:
            den +=1
    nouse.append(no)
    using.append(use)
    ben.append(den)

    

In [238]:
print(len(nouse), len(using), len(ben))

2491 2491 2491


In [239]:
len(amount)

2491

In [240]:
df = pd.DataFrame({'이름':name, '사용가능시간':time,'구분':typ, '사용가능':nouse,'사용중':using,'사용불가':ben,'도로명주소':add,'연락처':phone,'충전요금':fee, '급속충전량':amount,'참고사항':etc})
df.head()

,이름,사용가능시간,구분,사용가능,사용중,사용불가,도로명주소,연락처,충전요금,급속충전량,참고사항
0,DMC 산학협력연구센터,24시간 이용가능,급속,1,0,0,서울특별시 마포구 매봉산로 37,1661-9408,유료,급속(100kW멀티),10분/500원
1,Park M (주차복합빌딩),24시간 이용가능,급속,2,0,0,서울특별시 마포구 매봉산로 80,1661-9408,무료,급속(200kW동시),"최초30분 1500원, 10분당 500원 주민센터이용 50% 할인"
2,SETEC,24시간 이용가능,급속,1,0,0,서울특별시 강남구 남부순환로3104,1661-9408,유료,급속(100kW멀티),"충전차량에한하여 1시간무료 10분당500월 일주차15,000원"
3,sh서울주택도시공사,24시간 이용가능,급속,2,0,0,서울특별시 강남구 개포로 621,1661-9408,유료,급속(200kW동시),없음
4,가로공원로 지하공영주차장,24시간 이용가능,급속,1,0,0,서울특별시 강서구 가로공원로 189 지하주차장 05,1661-9408,유료,급속(50kW),주차요금 별도


In [242]:
df

,이름,사용가능시간,구분,사용가능,사용중,사용불가,도로명주소,연락처,충전요금,급속충전량,참고사항
0,DMC 산학협력연구센터,24시간 이용가능,급속,1,0,0,서울특별시 마포구 매봉산로 37,1661-9408,유료,급속(100kW멀티),10분/500원
1,Park M (주차복합빌딩),24시간 이용가능,급속,2,0,0,서울특별시 마포구 매봉산로 80,1661-9408,무료,급속(200kW동시),"최초30분 1500원, 10분당 500원 주민센터이용 50% 할인"
2,SETEC,24시간 이용가능,급속,1,0,0,서울특별시 강남구 남부순환로3104,1661-9408,유료,급속(100kW멀티),"충전차량에한하여 1시간무료 10분당500월 일주차15,000원"
3,sh서울주택도시공사,24시간 이용가능,급속,2,0,0,서울특별시 강남구 개포로 621,1661-9408,유료,급속(200kW동시),없음
4,가로공원로 지하공영주차장,24시간 이용가능,급속,1,0,0,서울특별시 강서구 가로공원로 189 지하주차장 05,1661-9408,유료,급속(50kW),주차요금 별도
...,...,...,...,...,...,...,...,...,...,...,...
2486,홍로공영주차장,24시간 이용가능,급속,0,1,0,제주특별자치도 서귀포시 동홍동 1480-6,1661-9408,무료,급속(100kW멀티),"주간 유료, 야간 무료"
2487,화북동주민센터,24시간 이용가능,급속,0,1,0,제주특별자치도 제주시 화북1동 1976-6,1661-9408,유료,급속(50kW),주차요금 별도
2488,환경성질환예방관리센터,24시간 이용가능,급속,1,0,0,제주특별자치도 제주시 구좌읍 다랑쉬북로 68-92 환경성질환예방관리센터(평대리 2981),1661-9408,유료,급속(50kW),없음
2489,회수사거리 인근 공영주차장,24시간 이용가능,급속,0,1,0,제주특별자치도 서귀포시 회수동 788-2,1661-9408,유료,급속(50kW),주차요금 별도
